In [1]:
import json
import csv
import os
import matplotlib.pyplot as plt

In [2]:
# !ls "myLabels/"
# !ls "labels_1125/"

In [3]:
root = "/home/arthurc/labels_1202/"
flaw_json_list = ["benchpress_flaw", "deadlift_flaw", "squat_flaw"]
flaw_dict = {}

for s in flaw_json_list:
    with open(root + s + ".json") as f:
        flaw_json = json.load(f)
        
    flaw_dict[s] = []
#     print(flaw_json)
    for val in flaw_json["data"]:
#         print(val)
        lt = val["gt_labels"]["lift_type"]
        great_one_flag = False
        for k, v in lt.items():
            if "great one" in v:
                great_one_flag = True
        
        if great_one_flag:
            temp_dict = {}
            temp_dict["filename"] = val["metadata"]["filename"]
            temp_dict["folder"] = val["metadata"]["folder"]
            temp_dict["macro_label"] = val["metadata"]["macro_label"]
            flaw_dict[s].append(temp_dict)
        
# print(flaw_list)
print(json.dumps(flaw_dict, sort_keys=True, indent=4))

save_file_path = root + "great_ones.json"

os.makedirs(os.path.dirname(save_file_path), exist_ok=True)
with open(save_file_path, 'w') as f:
    json.dump(flaw_dict, f, indent=4, separators=(',', ': '), sort_keys=True)

{
    "benchpress_flaw": [
        {
            "filename": "36708735_2163142547298807_1206170208494944256_n",
            "folder": "benchpress",
            "macro_label": "benchpress"
        },
        {
            "filename": "36333335_244061039519059_7891191769977585664_n",
            "folder": "squat",
            "macro_label": "benchpress"
        },
        {
            "filename": "36378949_420094115157808_84955767374872576_n",
            "folder": "benchpress",
            "macro_label": "benchpress"
        },
        {
            "filename": "35813616_2071046879783824_5704623782145032192_n",
            "folder": "benchpress",
            "macro_label": "benchpress"
        },
        {
            "filename": "36003403_1036444189845683_916060790563736583_n",
            "folder": "benchpress",
            "macro_label": "benchpress"
        },
        {
            "filename": "35935762_835868019939525_5036015292484943872_n",
            "folder": "benchpress",
   

In [4]:
openpose_path = "/data0/Results/lifting_openpose/"
openpose_folder_list = ["squat", "benchpress", "deadlift"]

squat_rep_json_path = "/home/dennyh/squat_reps.json"

lt = "squat"

openpose_json_path = openpose_path + lt + "/"
file_list = next(os.walk(openpose_json_path))[1]

flaw_classify_dict = {}

with open(squat_rep_json_path) as f:
    squat_rep_json = json.load(f)

for file in squat_rep_json.keys():
    video_name = file
    rep_seg = squat_rep_json[video_name]
    flaw_classify_dict[video_name] = []
    for d in rep_seg:
        frame_list = [d["start"], d["mid"]]
        start_bottom_frame = []

        left_side_flag = True
        no_depth_flag = False
        
        temp_dict = {}

        for i, frame in enumerate(frame_list, start = 0):
            openpose_frame_path = openpose_json_path + video_name + "/" + video_name + "_" + format(frame, '012d') + "_keypoints.json"

            with open(openpose_frame_path) as f:
                openpose_json = json.load(f)

            kp_2d = openpose_json["people"][0]["pose_keypoints_2d"]

            useful_kp = {}
            useful_kp["LHip"] = kp_2d[(12) * 3 : (12 + 1) * 3]
            useful_kp["RHip"] = kp_2d[(9) * 3 : (9 + 1) * 3]
            useful_kp["LKnee"] = kp_2d[(13) * 3 : (13 + 1) * 3]
            useful_kp["RKnee"] = kp_2d[(10) * 3 : (10 + 1) * 3]

            L_conf_sum = useful_kp["LHip"][2] + useful_kp["LKnee"][2]
            R_conf_sum = useful_kp["RHip"][2] + useful_kp["RKnee"][2]

            if L_conf_sum >= R_conf_sum and i == 0:
                left_side_flag = True
            elif L_conf_sum < R_conf_sum and i == 0:
                left_side_flag = False

            if left_side_flag == True:
                if useful_kp["LHip"][1] <= useful_kp["LKnee"][1] and i == 1:
                    no_depth_flag = True
            else:
                if useful_kp["RHip"][1] <= useful_kp["RKnee"][1] and i == 1:
                    no_depth_flag = True
        
        flaw_classify_dict[video_name].append(no_depth_flag)

print(json.dumps(flaw_classify_dict, sort_keys=True, indent=4))

{
    "29016345_199685493964578_7521323243834824401_n": [
        false
    ],
    "30213426_162993557719694_8182691775804906491_n": [
        false,
        false,
        true,
        false,
        false
    ],
    "35008537_213435235971618_5522297846289661952_n": [
        false,
        true
    ],
    "35116786_266295277463196_7770498034999230464_n": [
        true
    ],
    "35167494_225177954760040_7807092771268853760_n": [],
    "35169335_453748378384522_771227524788125696_n": [
        true,
        true,
        false,
        false,
        false,
        false,
        false
    ],
    "35451663_2217386021611018_2545453943257825280_n": [
        true
    ],
    "35505433_240571363201487_7690057034166697984_n": [
        false
    ],
    "35542376_196040734441794_6539317325701578752_n": [
        false,
        false,
        false,
        true,
        false,
        false
    ],
    "35549879_1603397959785838_5875048874724294656_n": [
        true,
        false,
     

In [5]:
squat_gt_labels_path = "/home/arthurc/labels_1202/squat_flaw.json"

with open(squat_gt_labels_path) as f:
    squat_gt_labels = json.load(f)

true_positive_count = 0
false_positive_count = 0
true_negative_count = 0
false_negative_count = 0

result_dict = {}
result_dict["TP"] = []
result_dict["TN"] = []
result_dict["FP"] = []
result_dict["FN"] = []

for fn, no_depth_list in flaw_classify_dict.items():
    for squat_gt in squat_gt_labels["data"]:
        if squat_gt["metadata"]["filename"] == fn and no_depth_list != []:
            
            temp_dict = {}
            temp_dict["filename"] = squat_gt["metadata"]["filename"]
            temp_dict["folder"] = squat_gt["metadata"]["folder"]
            temp_dict["macro_label"] = squat_gt["metadata"]["macro_label"]
            
            if "no depth" in squat_gt["gt_labels"]["lift_type"]["bottom"]:
                if no_depth_list[-1]:
#                 if True in no_depth_list:
                    true_positive_count += 1
                    result_dict["TP"].append(temp_dict)
        
                else:
                    false_negative_count += 1
                    result_dict["FN"].append(temp_dict)
                    
            else:
                if not no_depth_list[-1]:
#                 if not True in no_depth_list:
                    true_negative_count += 1
                    result_dict["TN"].append(temp_dict)
        
                else:
                    false_positive_count += 1
                    result_dict["FP"].append(temp_dict)

print(json.dumps(result_dict, sort_keys=True, indent=4))
                
print((true_positive_count + true_negative_count) / (true_positive_count + false_negative_count + true_negative_count + true_positive_count))
print()
print(true_positive_count, false_positive_count)
print(false_negative_count, true_negative_count)

save_file_path = root + "flaw_classifier_result.json"

os.makedirs(os.path.dirname(save_file_path), exist_ok=True)
with open(save_file_path, 'w') as f:
    json.dump(result_dict, f, indent=4, separators=(',', ': '), sort_keys=True)

{
    "FN": [
        {
            "filename": "36441248_189385255089174_9202051522844688384_n",
            "folder": "squat",
            "macro_label": "squat"
        },
        {
            "filename": "35808651_187188368617555_5832283743624101888_n",
            "folder": "benchpress",
            "macro_label": "squat"
        },
        {
            "filename": "36583400_2131517937105279_4826213235361316864_n",
            "folder": "squat",
            "macro_label": "squat"
        },
        {
            "filename": "36519070_199580920746807_7629294173325099008_n",
            "folder": "benchpress",
            "macro_label": "squat"
        },
        {
            "filename": "36591257_633256053689966_2890508532031946752_n",
            "folder": "squat",
            "macro_label": "squat"
        },
        {
            "filename": "36444621_675912852741341_2544324087686103040_n",
            "folder": "squat",
            "macro_label": "squat"
        },
        {

In [6]:
openpose_path = "/data0/Results/lifting_openpose/"
openpose_folder_list = ["squat", "benchpress", "deadlift"]

squat_rep_json_path = "/home/dennyh/squat_reps.json"

lt = "squat"

openpose_json_path = openpose_path + lt + "/"
file_list = next(os.walk(openpose_json_path))[1]

flaw_classify_dict = {}

with open(squat_rep_json_path) as f:
    squat_rep_json = json.load(f)

for file in squat_rep_json.keys():
    video_name = file
    rep_seg = squat_rep_json[video_name]
    flaw_classify_dict[video_name] = {}
    for j, d in enumerate(rep_seg, start = 1):
        frame_list = [d["start"], d["mid"]]
        start_bottom_frame = []

        left_side_flag = True
        no_depth_flag = False
        
        temp_dict = {}

        for i, frame in enumerate(frame_list, start = 0):
            openpose_frame_path = openpose_json_path + video_name + "/" + video_name + "_" + format(frame, '012d') + "_keypoints.json"

            with open(openpose_frame_path) as f:
                openpose_json = json.load(f)

            kp_2d = openpose_json["people"][0]["pose_keypoints_2d"]

            useful_kp = {}
            useful_kp["LHip"] = kp_2d[(12) * 3 : (12 + 1) * 3]
            useful_kp["RHip"] = kp_2d[(9) * 3 : (9 + 1) * 3]
            useful_kp["LKnee"] = kp_2d[(13) * 3 : (13 + 1) * 3]
            useful_kp["RKnee"] = kp_2d[(10) * 3 : (10 + 1) * 3]

            L_conf_sum = useful_kp["LHip"][2] + useful_kp["LKnee"][2]
            R_conf_sum = useful_kp["RHip"][2] + useful_kp["RKnee"][2]

            if L_conf_sum >= R_conf_sum and i == 0:
                left_side_flag = True
            elif L_conf_sum < R_conf_sum and i == 0:
                left_side_flag = False

            if left_side_flag == True:
                if useful_kp["LHip"][1] <= useful_kp["LKnee"][1] and i == 1:
                    no_depth_flag = True
            else:
                if useful_kp["RHip"][1] <= useful_kp["RKnee"][1] and i == 1:
                    no_depth_flag = True
        
        if no_depth_flag:
            flaw_classify_dict[video_name]["Rep " + str(j)] = "No depth"
        else:
            flaw_classify_dict[video_name]["Rep " + str(j)] = "Flawless"

print(json.dumps(flaw_classify_dict, sort_keys=True, indent=4))

{
    "29016345_199685493964578_7521323243834824401_n": {
        "Rep 1": "Flawless"
    },
    "30213426_162993557719694_8182691775804906491_n": {
        "Rep 1": "Flawless",
        "Rep 2": "Flawless",
        "Rep 3": "No depth",
        "Rep 4": "Flawless",
        "Rep 5": "Flawless"
    },
    "35008537_213435235971618_5522297846289661952_n": {
        "Rep 1": "Flawless",
        "Rep 2": "No depth"
    },
    "35116786_266295277463196_7770498034999230464_n": {
        "Rep 1": "No depth"
    },
    "35167494_225177954760040_7807092771268853760_n": {},
    "35169335_453748378384522_771227524788125696_n": {
        "Rep 1": "No depth",
        "Rep 2": "No depth",
        "Rep 3": "Flawless",
        "Rep 4": "Flawless",
        "Rep 5": "Flawless",
        "Rep 6": "Flawless",
        "Rep 7": "Flawless"
    },
    "35451663_2217386021611018_2545453943257825280_n": {
        "Rep 1": "No depth"
    },
    "35505433_240571363201487_7690057034166697984_n": {
        "Rep 1": "F